<a href="https://colab.research.google.com/github/edki96/Kio-Mugweru-Projects/blob/main/Account_Allocation_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas openpyxl schedule holidays

In [ ]:
import pandas as pd
import schedule
import time
from datetime import datetime, timedelta
from openpyxl import load_workbook
import holidays
import os

# Constants
FILE = 'Accounts_for_testing.xlsx'
CUSTOMER_SHEET = 'Customers'
MANAGER_SHEET = 'Agents'
LOG_SHEET = 'AssignmentLog'
KENYAN_HOLIDAYS = holidays.Kenya()
MAX_PER_MANAGER = 500
MAX_MONTHS = 3

# Load Excel Data
def load_data():
    customers = pd.read_excel(FILE, sheet_name=CUSTOMER_SHEET)
    managers = pd.read_excel(FILE, sheet_name=MANAGER_SHEET)
    try:
        log = pd.read_excel(FILE, sheet_name=LOG_SHEET)
    except:
        log = pd.DataFrame(columns=['Customer_ID', 'Manager_ID', 'FirstAssigned', 'LastContacted', 'LastPosition'])
    return customers, managers, log

# Check if today is valid
def is_valid_day(today):
    return today.weekday() != 6 and today not in KENYAN_HOLIDAYS



In [ ]:
# Filter eligible accounts
def get_eligible_accounts(customers, log, today):
    merged = customers.merge(log, on='Customer_ID', how='left')
    merged['FirstAssigned'] = pd.to_datetime(merged['FirstAssigned'], errors='coerce')
    merged['LastContacted'] = pd.to_datetime(merged['LastContacted'], errors='coerce')
    eligible = []

    for _, row in merged.iterrows():
        acc_id = row['Customer_ID']
        first = row['FirstAssigned']
        last = row['LastContacted']

        # New account
        if pd.isna(first):
            eligible.append((acc_id, None, None))
        # Within 3 months and not yet contacted today
        elif (today - first.date()) < timedelta(days=MAX_MONTHS * 30):
            if pd.isna(last) or last.date() != today:
                eligible.append((acc_id, row['Manager_ID'], row['LastPosition']))
    return eligible

# Assign accounts
def allocate_accounts(eligible_accounts, managers, today):
    allocations = []
    manager_ids = list(managers['Manager_ID'])
    manager_counts = {mid: 0 for mid in manager_ids}
    acc_queue = []

    # Ensure top-to-bottom by row order (same order as customer list)
    for acc in eligible_accounts:
        acc_queue.append(acc)

    for acc_id, prev_manager, prev_pos in acc_queue:
        assigned = False

        # Reuse previous manager if within limits
        if prev_manager and manager_counts[prev_manager] < MAX_PER_MANAGER:
            allocations.append({'Customer_ID': acc_id, 'Manager_ID': prev_manager, 'AllocationDate': today})
            manager_counts[prev_manager] += 1
            assigned = True
        elif not assigned:
            # Pick first manager with available quota
            for mid in manager_ids:
                if manager_counts[mid] < MAX_PER_MANAGER:
                    allocations.append({'Customer_ID': acc_id, 'Manager_ID': mid, 'AllocationDate': today})
                    manager_counts[mid] += 1
                    break
        # Stop if all managers full
        if all(v >= MAX_PER_MANAGER for v in manager_counts.values()):
            break
    return pd.DataFrame(allocations)

# Update log
def update_log(log, allocations):
    today = allocations['AllocationDate'].iloc[0]
    for _, row in allocations.iterrows():
        acc_id = row['Customer_ID']
        manager = row['Manager_ID']
        existing = log[log['Customer_ID'] == acc_id]

        if existing.empty:
            log = pd.concat([log, pd.DataFrame([{
                'Customer_ID': acc_id,
                'Manager_ID': manager,
                'FirstAssigned': today,
                'LastContacted': today,
                'LastPosition': None
            }])])
        else:
            log.loc[log['Customer_ID'] == acc_id, 'LastContacted'] = today
    return log

# Report
def generate_report(allocations):
    return allocations.groupby('Manager_ID').size().reset_index(name='AccountsAssigned')

# Save
def save_to_excel(allocations, report, log, today):
    date_str = today.strftime('%d%m%Y')
    with pd.ExcelWriter(FILE, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        allocations.to_excel(writer, sheet_name=date_str, index=False)
        report.to_excel(writer, sheet_name=f'Report_{date_str}', index=False)
        log.to_excel(writer, sheet_name=LOG_SHEET, index=False)

# Main Job
def run_allocation(initial_run=False):
    today = datetime.today().date()
    print(f"▶ Running allocation for {today}")

    if not is_valid_day(today) and not initial_run:
        print("⛔ Not a working day in Kenya.")
        return

    customers, managers, log = load_data()
    eligible = get_eligible_accounts(customers, log, today)
    if not eligible:
        print("⚠ No eligible accounts found.")
        return

    allocations = allocate_accounts(eligible, managers, today)
    updated_log = update_log(log, allocations)
    report = generate_report(allocations)
    save_to_excel(allocations, report, updated_log, today)
    print(f"✅ Allocated {len(allocations)} accounts.")

# First run immediately
run_allocation(initial_run=True)

# Schedule every day at 7AM
schedule.every().day.at("07:00").do(run_allocation)

print("⏰ Scheduled to run daily at 07:00...")

while True:
    schedule.run_pending()
    time.sleep(60)


▶ Running allocation for 2025-07-18
✅ Allocated 20500 accounts.
⏰ Scheduled to run daily at 07:00...
